In [1]:
import spelling_bee

In [2]:
official_grid_text = """
         4	  5	  6	  7	  8	  9	  Σ
A:	  -	  -	  -	  -	  1	  -	  1
C:	  2	  1	  -	  -	  -	  -	  3
H:	  1	  1	  -	  -	  1	  -	  3
I:	  1	  -	  -	  -	  -	  -	 1
M:    7	  5	  2	  1	  2	  1	 18
T:	  2	  -	  3	  1	  -	  -	  6
Σ:	 13	  7	  5	  2	  4	  1	 32

Two letter list:

AR-1
CA-1 CH-1 CR-1
HA-3
IM-1
MA-14 MI-4
TA-3 TR-3"""

In [3]:
paste = """
Cami
Charm
Cram
Harm
Imam
Maim
Mama
Mamma
Mart
Match
Matcha
Math
Matriarch
Mica
Mirth
Mitt
Tram
Trim
"""

In [5]:
bee = spelling_bee.SpellingBee()
# bee.read_found_words(paste)
# bee.read_official_grid(official_grid_text)

# bee.print_counts()
# print()
# bee.print_grid_comparison()
# print()
# bee.print_two_letter_list_comparison()
bee.import_puzzle()
bee.print_official_counts()

- Official Grid
    4  5  6  7  8   Σ
A:  2  -  1  1  3   7
C:  -  -  -  -  1   1
I:  -  -  1  -  -   1
Q:  1  2  2  -  1   6
T:  3  3  -  -  -   6
U:  1  1  -  -  -   2
Σ:  7  6  4  1  5  23

AC-2 AQ-4 AU-1 
CI-1 
IN-1 
QU-6 
TA-2 TU-4 
UN-2 
